In [ ]:
%tensorflow_version 2.x

In [ ]:
%pylab inline

In [ ]:
from tensorflow import keras

In [ ]:
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
#Create the encoder model
encoder = keras.models.Sequential([
    #Flatten the iput images (28x28) into a 1D array of 784 el3ments
    keras.layers.Flatten(input_shape= [28, 28]),
    #Add a dense layer with 100 neurons and ReLU activation function
    keras.layers.Dense(100, activation="relu"),
    #Add another dense layer with 30 neurons and ReLU acivation function
    keras.layers.Dense(30, activation = "relu"),
])

In [ ]:
#Create the decoder model
decoder = keras.models.Sequential([
    #Add a dense layer with 100 neurons and ReLU activation function, input shape is
    keras.layers.Dense(100, activation="relu", input_shape= [30]),
    #Add a dense layer with 784 neurons (28*28) and sigmoid activation function function to ......
    keras.layers.Dense(28 * 28, activation = "sigmoid"),
    #Reshape the output into the original image shape (28 x 28)
    keras.layers.Reshape([28, 28])
])

In [ ]:
stacked_autoencoder = keras.models.Sequential([encoder, decoder])

In [ ]:
#Compile the stacked autoencoder model 
stacked_autoencoder.compile(
    #Use binary cross_entropy loss function for binary imput data (normalized pixel)
    loss = "binary_crossentropy",
    #Use the Adam optimizer for efficient training
    optimizer = 'adam'
)

In [ ]:
#Train the stacked autoencoder model
history = stacked_autoencoder.fit(
    #Input and target data are both x_train (input images ) for reconstruction
    x_train, x_train,
    #Train for 10 epochs
    epochs = 10,
    #Use x_test for validation during training
    validation_data = [x_test, x_test]

)

In [ ]:
#Set the figure size for the plot
figsize(20, 5)

#Iterate over a range of 8 examples from the test set
for i in range(8):
  #plot the original image from the test set 
  subplot(2, 8, i+1)
  #Make a prediction using the stacked autoencoder on the current test image
  pred = stacked_autoencoder.predict(x_test[i].reshape((1, 28, 28)))
  # Display the original image
  imshow(x_test[i], cmap = "binary")
  #plot the reconstructed image by the stacked autoencoder
  subplot(2, 8, i+8+1)
  # Display the reconstructed image
  imshow(pred.reshape((28, 28)), cmap = "binary")

In [ ]:
figsize(10, 5)
# Plot the original image
subplot(1, 3, 1)
imshow(x_test[i], cmap = "binary")

# Plot the latent vector representation obtained from the encoder
subplot(1, 3, 2)
# Predict the latent vector representation of the selected test image
latent_vector = encoder.predict(x_test[i].reshape((1, 28, 28)))
imshow(latent_vector, cmap = "binary")

# Plot the reconstructed image from the latent vector using the decoder
subplot(1, 3, 3)
# Reconstruct the image from the latent vector representation 
pred = decoder.predict(latent_vector)
imshow(pred.reshape((28, 28)), cmap = "binary")

In [ ]:
sparsity_low = 30 / (28 * 28)  # Upper bound for sparsity constraints
#sparsity_high = 1 - sparsity_low  # Lower bound for sparsity constraint
sparsity_high = 1 - 30 / (28 * 28)

# Print both values
print(sparsity_low, sparsity_high)

In [ ]:
# Define the encoder model using convolutional layers
encoder = keras.models.Sequential([
    # Reshpe the input into a 28 x 28 x 1 image (grayscale)
    keras.layers.Reshape([28, 28, 1], input_shape=[28, 28]),
    # First convolutional layer with 16 filters, each of size 3x3, ReLU activation, 
    keras.layers.Conv2D(16, kernel_size = (3, 3), padding = "same", activation= "relu"), 
    # Max pooling layer with pool size 2x2 to dowsample the spatial dimensions
    keras.layers.MaxPool2D(pool_size=2),
    # Second convolutional layer with 32 filters, each of size 3x3, ReLU activation
    keras.layers.Conv2D(32, kernel_size=(3, 3), padding= "same", activation= "relu"),
    #Max Pooling layer
    keras.layers.MaxPool2D(pool_size=2),
    # Third consvolutional layer with 64 filters, each of size 3x3, Relu activation
    keras.layers.Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu"),
    #Max Pooling layer
    keras.layers.MaxPool2D(pool_size=2)
])

In [ ]:
# Define the decoder model using convolutional transpose layers
decoder = keras.models.Sequential([
    # Convolutional transpose layer with 32 filters, eacho of size 3x3 stride 2, Relu
    keras.layers.Conv2DTranspose(16, kernel_size=(3,3), strides=2, padding="valid", 
                                 activation="relu", input_shape=[3, 3, 64]),
    # Convolutional transpose layer with 16 filters, each of size 3x3, stride 2, ReLU
    keras.layers.Conv2DTranspose(16, kernel_size=(3, 3), strides=2, padding="same",
                                 activation="relu"),
    # Convolutional transpose layer with 1 filter, size 3x3, stride2, sigmoid activat
    keras.layers.Conv2DTranspose(1, kernel_size=(3, 3), strides=2, padding="same",
                                 activation="sigmoid"),
    # Reshape the output into the original image shape (28x28)
    keras.layers.Reshape([28, 28])
]
)

In [ ]:
# Stack the encoder and decoder models to create the stacked autoencoder
stacked_autoencoder = keras.models.Sequential([encoder, decoder])

In [ ]:
# Compile the stacked autoencoder model
stacked_autoencoder.compile(
    # Use binary cross_entropy loss function for binary input data (normalized Pixel)
    loss = "binary_crossentropy",
    # Use the Adam optimazer for efficient training
    optimizer = 'adam'
    )

In [ ]:
# Train the stacked autoencoder model on the training data
history = stacked_autoencoder.fit(
    # Input and target data are both x_train (input images) for reconstruction
    x_train, x_train,
    # Train for 10 epochs
    epochs = 10,
    # Use x_test for validation during training
    validation_data= [x_test, x_test]
)

In [ ]:
# Set the figure size for the plot
figsize(20, 5)

# Iterate over 8 examples from the test set
for i in range(8):
  # plot the original image from the test set
  subplot(2, 8, i+1)
  # Make a prediction using the stacked autoencoder on the current test image
  pred = stacked_autoencoder.predict(x_test[i].reshape((1, 28, 28)))
  # Display the original image
  imshow(x_test[i], cmap= "binary")
  # Plot the reconstructed image by the stacked autoencoder
  subplot(2, 8, i+8+1)
  # Display the reconstructed image
  imshow(pred.reshape((28, 28)), cmap="binary")

In [ ]:
# SEt the figure size for the plot
figsize(15, 15)

# Iterate over all filters in the last convolutional layer of the encoder
for i in range(8 * 8):
  # Plot each filter as a subplot in an 8x8 grid
  subplot(8, 8, i+1)
  # Display the weights (filters) of the convolutional layer
  imshow (encoder.layers[-2].weights[0][:, :, 0, i])

In [ ]:
Denoising Autoencoder
import numpy as np

In [ ]:
# Set the figure size for the plot
figsize(5, 10)

# Plot the original image from the test set
subplot(1, 2, 1)
imshow(x_test[0], cmap="binary")

# Plot the noisy version of the original image
subplot(1, 2, 2)
# GEnerate random noise and add it to the original image
noise = np.random.random((28, 28,)) /4
imshow(x_test[0] + noise, cmap= "binary")

In [ ]:
# Define the encoder model usin dense (fully connected) layers
encoder = keras.models.Sequential([
    # Flatten the input images (28x28) into a 1D array of 784 elements
    keras.layers.Flatten(input_shape=[28, 28]),
    # Add a dense layer with 100 neurons and ReLU activation function
    keras.layers.Dense(100, activation="relu"),
    #Add another dense laher with 100 neurons and ReLU activation 
    keras.layers.Dense(100, activation="relu"),
    # Add a dense layer with 30 neurons and ReLU activation function
    keras.layers.Dense(30, activation="relu")
])

In [ ]:
# Define the decoder model using dense (fully connected) layers
decoder = keras.models.Sequential([
    # Add a dense layer with 100 neurons and ReLu activation function,
    keras.layers.Dense(100, activation="relu", input_shape=[30]),
    # Add another dense layer with 100 neurons and Relu activation
    keras.layers.Dense(100,activation="relu"),
    # Add a dense layer with 784 neurons (28* 28) and sigmoid activatation function
    keras.layers.Dense(28 * 28, activation='sigmoid'),
    # Reshape the output into the original image shape (28x28)
    keras.layers.Reshape([28, 28])
]
)

In [ ]:
# Stack the encoder and decoder models to create the stacked autoencoder
stacked_autoencoder = keras.models.Sequential([encoder, decoder])

In [ ]:
# Compile the stacked autoencoder model
stacked_autoencoder.compile(
    # Use binary cross_entropy loss function for binary input data (normalized Pixel)
    loss = "binary_crossentropy",
    # Use the Adam optimazer for efficient training
    optimizer = 'adam'
    )

In [ ]:
# Add random noise to the training and test data
x_train_noise = x_train + ((np.random.random(x_train.shape)) / 4)
x_test_noise = x_test + ((np.random.random(x_test.shape)) / 4)

In [ ]:
# Display an example of the noisy training data
imshow(x_train_noise[0], cmap='binary')

In [ ]:
# Train the stacked autoencoder model on the noisy training data
history = stacked_autoencoder.fit(
    # Input is the noisy training data (x_train_noise), target is the original clean
    x_train_noise, x_train,
    # Train for 10 epochs
    epochs = 10,
    # Use noisy test data (x_test_noise) for validation during  during training, with original 
    validation_data = [x_test_noise, x_test]
)